In [355]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup as bs

In [363]:
def get_CV(contents):
    data_columns = ['nr','amet', 'tööandja','asukoht', 'sisestatud','aegub','kuupalk']
    
    CV_data = pd.DataFrame(columns = data_columns, index = range(0,1))
    CV_table = pd.DataFrame(columns = data_columns)
   
    try:
        soup = bs(contents.text, 'html.parser')
        rows = soup.findAll("div", {"class":"offer_primary_info"})
    except:
        print('Could not parse web contents')
        return CV_table
    
    i = 0
    for row in rows:
        amet = row.find("a", {"itemprop":"title"}).text
        tööandja = row.find("a", {"itemprop":"name"}).text

        if row.find("a", {"itemprop":"address"}) is None:
            asukoht =  row.find("span", {"itemprop":"jobLocation"}).text
        else:
            asukoht = row.find("a", {"itemprop":"address"}).text
        
        sisestatud = row.find("span", {"itemprop":"datePosted"}).text
        aegub = row.find("ul", {"class":"cvo_module_offer_meta offer_dates"}).findAll("li")[1].text
        
        try:
            kuupalk = row.find("span", {"class":"salary-blue"}).text
        except:
            kuupalk = "puudub"
        
        # formatting
        sisestatud = re.split("\s", sisestatud)[0]
        aegub = re.split("\s", aegub)[1]
        kuupalk = kuupalk.replace('Kuupalk: ', '')
        
         # dataframe
        CV_data['nr'] = i
        CV_data['amet'] = amet
        CV_data['tööandja'] = tööandja
        CV_data['asukoht'] = asukoht
        CV_data['sisestatud'] = sisestatud
        CV_data['aegub'] = aegub
        CV_data['kuupalk'] = kuupalk

        CV_table = CV_table.append(CV_data)
        i += 1
        
    message = ' Downloaded ' + str(len(CV_table)) + ' lines of CV data'
    print(message)
    
    CV_table = CV_table.set_index('nr')

    return CV_table

In [364]:
link1 = 'https://www.cv.ee/toopakkumised/q-analüüs/q-analüütik'
contents = requests.get(link1)
first_page = get_CV(contents)
first_page.head()

 Downloaded 22 lines of CV data


,amet,tööandja,asukoht,sisestatud,aegub,kuupalk
nr,,,,,,
0,Analüütik (väliskaubandusstatistika),Statistikaamet,Tallinn,4,16.10.2019,puudub
1,ANALÜÜTIK-EELARVESTAJA,Kaitseministeerium,Järve tn 34A,15,14.10.2019,puudub
2,MÜÜGIJUHT,ruut8,Lõõtsa 8a,17,31.10.2019,puudub
3,Analüütik,Statistikaamet,Tallinn,21,16.10.2019,puudub
4,MÜÜGISPETSIALIST,Pharmanet OÜ,Tallinn,1,20.10.2019,puudub
